In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [6]:
#Set up...
confthreshold =0.5
nmsthreshold = 0.3
modelconfiguration='yolov4.cfg'
modelweight='yolov4.weights'
classfile='coco.names'
classnames=[]
with open(classfile,'rt') as f:
    classnames=f.read().rstrip('\n').split('\n')
net = cv2.dnn.readNet(modelweight, modelconfiguration)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [7]:
def findObjects(outputs,img):
    hT,wT,cT=img.shape
    boundbox=[]
    classIDs=[]
    confs=[]
    #detect object,compuate and find the max score for each.
    for output in outputs:
        for dect in output:
            scores = dect[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confthreshold:
                w,h = int(dect[2]*wT),int(dect[3]*hT)
                x,y = int((dect[0]*wT)-w/2),int((dect[1]*hT)-h/2)
                boundbox.append([x,y,w,h])
                classIDs.append(classID)
                confs.append(float(confidence))
    indices=cv2.dnn.NMSBoxes(boundbox,confs,confthreshold,nmsthreshold)
    #draw bounding box and label output name
    for i in indices:
        i=i[0]
        box = boundbox[i]
        x,y,w,h = box[0],box[1],box[2],box[3]
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.putText(img,f'{classnames[classIDs[i]].upper()}{int(confs[i]*100)}%',
                   (x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)

In [15]:
#Take in an video source and label rect and names for each object.
def filedet(filename,wh):
    cap=cv2.VideoCapture(filename)
    prev_frame_time = 0
    new_frame_time = 0
    cv2.startWindowThread()
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    while cap.isOpened():
        success,img=cap.read()
        if success == True:
            blob=cv2.dnn.blobFromImage(img,1/255,(wh,wh),[0,0,0],1,crop=False)
            net.setInput(blob)
            layerNames= net.getLayerNames()
            outputNames = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
            outputs=net.forward(outputNames)
            findObjects(outputs,img)
            #find fps
            gray = img
            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time) 
            prev_frame_time = new_frame_time 
            fps = int(fps)
            fps = str(fps)
            cv2.putText(gray, fps, (7, 70), font, 1, (100, 255, 0), 3, cv2.LINE_AA)
            #open final window
            cv2.imshow('Img',img)
            #press q to quite the window
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [16]:
filedet(filename='conv2B test2.mp4',wh=320)